## First, import the libraries we will use

In [1]:
%matplotlib inline

import numpy as np
from datetime import datetime, timedelta
from pyproj import Proj
import xarray
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy import spatial

## Open the GOES-17 NetCDF File
Using xarray, I assign the opened file to the variable C for the CONUS domain.

In [2]:
#GOES 16 all in one file
#FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20190422102136_e20190422104509_c20190422105021.nc'

#GOES 17 on Amazon, each channel in a different file
FILE_5 ='OR_ABI-L2-CMIPC-M3C05_G17_s20190422102163_e20190422104536_c20190422105008.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190422102163_e20190422104536_c20190422104597.nc'
FILE_3 = 'OR_ABI-L2-CMIPM1-M3C03_G17_s20190461730268_e20190461730326_c20190461730375.nc'
File_5 = 'OR_ABI-L2-CMIPM1-M3C05_G17_s20190461730268_e20190461730326_c20190461730375.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190461732189_e20190461734562_c20190461735029.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190461732189_e20190461734562_c20190461735030.nc'
FILE_3 = 'OR_ABI-L2-CMIPM1-M3C03_G17_s20190461700268_e20190461700326_c20190461700370.nc'
FILE_5 = 'OR_ABI-L2-CMIPM1-M3C05_G17_s20190461700268_e20190461700326_c20190461700373.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190741802189_e20190741804562_c20190741805023.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190741802189_e20190741804562_c20190741805035.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190742102189_e20190742104562_c20190742105016.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190742102189_e20190742104562_c20190742105023.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190752002189_e20190752004562_c20190752005027.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190752002189_e20190752004562_c20190752005028.nc'
FILE_2 = 'OR_ABI-L2-CMIPC-M3C02_G17_s20190752002189_e20190752004562_c20190752005057.nc'

FILE_3_16 = 'OR_ABI-L2-CMIPC-M3C03_G16_s20190752002189_e20190752004562_c20190752005035.nc'
FILE_5_16 = 'OR_ABI-L2-CMIPC-M3C05_G16_s20190752002189_e20190752004562_c20190752005036.nc'

# goes 17
C_5 = xarray.open_dataset(FILE_5)
C_3 = xarray.open_dataset(FILE_3)
#channel 2 at finer spatial res
#C_2 = xarray.open_dataset(FILE_2)


In [3]:
#goes 17 info
C_5['goes_imager_projection']

<xarray.DataArray 'goes_imager_projection' ()>
array(-2147483647)
Coordinates:
    t        datetime64[ns] ...
    y_image  float32 ...
    x_image  float32 ...
Attributes:
    long_name:                       GOES-R ABI fixed grid projection
    grid_mapping_name:               geostationary
    perspective_point_height:        35786023.0
    semi_major_axis:                 6378137.0
    semi_minor_axis:                 6356752.31414
    inverse_flattening:              298.2572221
    latitude_of_projection_origin:   0.0
    longitude_of_projection_origin:  -137.0
    sweep_angle_axis:                x

In [4]:
# Satellite height goes 17
sat_h = C_5['goes_imager_projection'].perspective_point_height

# Satellite longitude
#sat_lon = C_5['goes_imager_projection'].longitude_of_projection_origin
#uncertainty whether at -137.2. -137 seems correct now
sat_lon = -137

# Satellite sweep
sat_sweep = C_5['goes_imager_projection'].sweep_angle_axis

# The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
# See details here: https://proj4.org/operations/projections/geos.html?highlight=geostationary
x = C_5['x'][:] * sat_h
y = C_5['y'][:] * sat_h

In [5]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

projection = ccrs.Geostationary(central_longitude=sat_lon, satellite_height=sat_h,
                               sweep_axis=sat_sweep)
img_extent = (x.min(), x.max(), y.min(), y.max())



#### The magic function: `pyproj.Proj`
This function creates a map projection object of the geostationary projection.

In [6]:
# Create a pyproj geostationary map object
p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep)

# Perform cartographic transformation for goes 17. That is, convert image projection coordinates (x and y)
# to latitude and longitude values.
XX, YY = np.meshgrid(x, y)
lons, lats = p(XX, YY, inverse=True)

print('GOES 17 dimensions: %s %s' % np.shape(lons))


GOES 17 dimensions: 3000 5000


In [7]:
lons_i = np.arange(-122.5, -111., 0.01)
lats_i = np.arange(35.5, 43., 0.01)
xx_i, yy_i = np.meshgrid(lons_i,lats_i)
dims_i = np.shape(xx_i)
iy = len(lats_i)
ix = len(lons_i)
lat_lon = np.dstack([lats.ravel(),lons.ravel()])[0]
print(np.shape(lat_lon))
print(lat_lon)
lat_lon_i = np.dstack([yy_i.ravel(),xx_i.ravel()])[0]
print(np.shape(lat_lon_i))
print(lat_lon_i)

(15000000, 2)
[[  53.51366846  175.58688993]
 [  53.51009397  175.62003143]
 [  53.50652579  175.65312979]
 ...
 [  14.80001316 -112.44766424]
 [  14.80021706 -112.43677203]
 [  14.80042107 -112.42587833]]
(862500, 2)
[[  35.5  -122.5 ]
 [  35.5  -122.49]
 [  35.5  -122.48]
 ...
 [  42.99 -111.03]
 [  42.99 -111.02]
 [  42.99 -111.01]]


In [8]:

print(np.shape(lats))
distance,index = spatial.KDTree(lat_lon).query(lat_lon_i)
print(distance)
print(index)

(3000, 5000)
[0.00677684 0.00730895 0.00726418 ... 0.00913847 0.00535514 0.00582392]
[5243745 5248746 5243746 ... 2634410 2639411 2639412]


In [9]:
np.save('proj_west_17',index)